In [ ]:
from numpy import sqrt, rint, zeros, random, zeros, ones

In [ ]:
N=10

In [ ]:
rx=zeros(N)

In [ ]:
rx=random.normal(0.5, 2, N)

In [ ]:
rx

In [ ]:
r2max=1.

In [ ]:
for k in range(N-1) :
    j=k+1
    #for j in range(k+1,N) :
    dx = rx[k]-rx[j:N]
    print(dx)
    input("Press Enter to continue...")
    dx[...]-= rint(dx)
    print(dx)
    input("Press Enter to continue...")
    r2 = dx*dx
    # using the mask array "b" for speedup
    b = r2 < r2max
    print(b)
    input("Press Enter to continue...")
    lm  = sqrt(r2[b])
    print('lm=', lm)
    #if lm<self.kg :
    for elm in lm :
        print(elm)

In [ ]:
N

In [ ]:
npmax=5
from numpy import cos, sin, pi

In [ ]:
cp = zeros((2*npmax+1,N))
cp[npmax,] = 1.0
sp = zeros((2*npmax+1,N))


In [ ]:
cp

In [ ]:
sp

In [ ]:
cp[npmax+1,] = cos(2*pi*rx)
sp[npmax+1,] = sin(2*pi*rx)

In [ ]:
cp

In [ ]:
sp

In [ ]:
for n in range(npmax-1):
    m = npmax+2+n
    print(n, m)
    cp[m,] = cp[npmax+1,]*cp[m-1,]-sp[npmax+1,]*sp[m-1,]
    sp[m,] = sp[npmax+1,]*cp[m-1,]+cp[npmax+1,]*sp[m-1,]

In [ ]:
cp

In [ ]:
sp

In [ ]:
for n in range(1,npmax+1):
    mm = npmax-n
    mp = npmax+n
    print(mp,mm)
    cp[mm,] = cp[mp,]
    sp[mm,] = -sp[mp,]

In [ ]:
cp

In [ ]:
sp